## Introdução
Este estudo tem por objetivo estabelecer uma correlação entre a taxa de aprocação/reprovação dos docentes do **Instituto Metrópole Digital** e as notas médias de suas avaliações como educadores.

## Base de dados

O Portal de Dados Abertos da UFRN tem como objetivo disponibilizar para toda a comunidade dados e informações públicas da Instituição, que podem ser usadas no desenvolvimento de aplicativos e ações.
Desse modo, todos os dados utilizados neste estudo podem ser verificados em : http://dados.ufrn.br/group/ensino


## Filtragem de dados

Tendo em vista a grande quantidade de informações disponibilizadas e as pequenas falhas encontradas em determinados pontos, uma filtragem dos dados se faz necessária.
Desse modo, algumas informações serão removidas do dataframe ao passo que outras serão mescladas.

## Importando bibliotecas

In [1]:
# importando as bibliotecas necessárias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Importando dados

In [7]:
#Avaliações de docência
nota_docente_url = 'https://bit.ly/2IqonoW'

#Componentes curriculares presenciais
disciplinas_url = 'http://bit.do/componentes_curriculares_ufrn'

#Turmas de 2015.1 até 2017.1
turmas_2017_1_url = 'https://bit.ly/2wn1AFI'
turmas_2016_2_url = 'https://bit.ly/2I14eGu'
turmas_2016_1_url = 'https://bit.ly/2IoXKAG'
turmas_2015_2_url = 'https://bit.ly/2wrhMpw'
turmas_2015_1_url = 'https://bit.ly/2IaxFC5'

#Matrículas em componentes de 2015.1 até 2017.1
notas_2017_1_url = 'https://bit.ly/2KMoTvS'
notas_2016_2_url = 'https://bit.ly/2rup6Ma'
notas_2016_1_url = 'https://bit.ly/2K7oy5w'
notas_2015_2_url = 'https://bit.ly/2rvknKb'
notas_2015_1_url = 'https://bit.ly/2I8YfeM'


#Dataframe das turmas
turmas_2017_1 = pd.read_csv(turmas_2017_1_url, sep=';')
turmas_2016_2 = pd.read_csv(turmas_2016_2_url, sep=';')
turmas_2016_1 = pd.read_csv(turmas_2016_1_url, sep=';')
turmas_2015_2 = pd.read_csv(turmas_2015_2_url, sep=';')
turmas_2015_1 = pd.read_csv(turmas_2015_1_url, sep=';')

#Dataframe das matriculas em componentes
notas_2017_1 = pd.read_csv(notas_2017_1_url, sep=';')
notas_2016_2 = pd.read_csv(notas_2016_2_url, sep=';')
notas_2016_1 = pd.read_csv(notas_2016_1_url, sep=';')
notas_2015_2 = pd.read_csv(notas_2015_2_url, sep=';')
notas_2015_1 = pd.read_csv(notas_2015_1_url, sep=';')


#Dataframe dos componentes curriculares presenciais
#realizado um filtro apenas por colunas mais importantes
disciplinas = pd.read_csv(disciplinas_url, sep=';', 
                          usecols=["id_componente","codigo","nivel","nome","unidade_responsavel"])

#Dataframe da avaliação dos docentes
#filtrando notas dos docentes
nota_docente = pd.read_csv(nota_docente_url, sep=';',
                           usecols=["id_docente", "nome_docente", "id_turma", "ano", 
                                    "autoavaliacao_aluno_media", "atuacao_profissional_media", 
                                    "postura_profissional_media" ])


## Preparação e Limpeza dos dados

### Turmas

In [48]:
#selecionando apenas as colunas necessárias
turmas_cols = ['id_turma', 'codigo_turma', 'id_docente_interno','id_componente_curricular', 'nivel_ensino',
               'local', 'ano', 'periodo', 'descricao_horario',  'capacidade_aluno', 'tipo',
               'data_consolidacao',  'situacao_turma', 'modalidade_participantes']


turmas_2015_1 = turmas_2015_1[turmas_cols]
turmas_2015_2 = turmas_2015_2[turmas_cols]
turmas_2016_1 = turmas_2015_1[turmas_cols]
turmas_2016_2 = turmas_2015_2[turmas_cols]
turmas_2017_1 = turmas_2015_1[turmas_cols]

#concatenando verticalmente todas as turmas em um so dataframe
frames = [turmas_2015_1, turmas_2015_2, turmas_2016_1, turmas_2016_2, turmas_2017_1]
turmas_total = pd.concat(frames)

# Adaptando o dataframe turma e disciplinas para o merge
# Esse procedimento é necessário pois o dataframe turma está codificado com ids
# A coluna id_componente precisa ser a mesma em ambos os dataframes
turmas_total.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)

# id_componente em ambos os dataframes precisam ser do mesmo tipo
turmas_total.id_componente = turmas_total.id_componente.astype(str)

# investigando os dados
turmas_total.head()

,id_turma,codigo_turma,id_docente_interno,id_componente,nivel_ensino,local,ano,periodo,descricao_horario,capacidade_aluno,tipo,data_consolidacao,situacao_turma,modalidade_participantes
0,1217099,01,18225.0,52922,LATO SENSU,Depto. de Oceanografia e Limnologia,2015,1,23456T234,26.0,REGULAR,2015-06-01 12:53:14.625,CONSOLIDADA,Presencial
1,1217100,01,18695.0,52921,LATO SENSU,Depto. de Oceanografia e Limnologia,2015,1,345T2345 6T234 (14/04/2015 - 17/04/2015),26.0,REGULAR,2015-10-08 16:19:21.901,CONSOLIDADA,Presencial
2,1217101,01,5753068.0,52925,LATO SENSU,Depto. de Oceanografia e Limnologia,2015,1,23456T123456,26.0,REGULAR,2015-06-01 12:50:38.962,CONSOLIDADA,Presencial
3,1217102,01,NaN,52920,LATO SENSU,Depto. de Oceanografia e Limnologia,2015,1,23456T123456,26.0,REGULAR,2015-08-26 11:48:34.146,CONSOLIDADA,Presencial
4,1217102,01,NaN,52920,LATO SENSU,Depto. de Oceanografia e Limnologia,2015,1,23456T123456,26.0,REGULAR,2015-08-26 11:48:34.146,CONSOLIDADA,Presencial


### Matrículas

In [49]:
#selecionando apenas as colunas necessárias
notas_cols = ['id_turma', 'discente', 'id_curso', 'unidade', 'nota', 'media_final', 'descricao']

notas_2015_1 = notas_2015_1[notas_cols]
notas_2015_2 = notas_2015_2[notas_cols]
notas_2016_1 = notas_2016_1[notas_cols]
notas_2016_2 = notas_2016_2[notas_cols]
notas_2017_1 = notas_2017_1[notas_cols]

#concatenando verticalmente
frames = [notas_2015_1, notas_2015_2, notas_2016_1, notas_2016_2, notas_2017_1]
notas_total = pd.concat(frames)

# Limpando o dataframe disciplinas com linhas desnecessárias
disciplinas.dropna(inplace=True)
disciplinas = disciplinas[~disciplinas.id_componente.duplicated()]

# investigando os dados
notas_total.head()

,id_turma,discente,id_curso,unidade,nota,media_final,descricao
0,57562481,8833c0e056f90497742882bfcc81b685,100300443.0,2.0,0.0,0.0,REPROVADO
1,57562481,8833c0e056f90497742882bfcc81b685,100300443.0,3.0,0.0,0.0,REPROVADO
2,57562481,7a0a661326418111f7d17aab098757aa,100300443.0,1.0,8.9,8.9,APROVADO
3,57562481,7a0a661326418111f7d17aab098757aa,100300443.0,2.0,8.9,8.9,APROVADO
4,57562481,7a0a661326418111f7d17aab098757aa,100300443.0,3.0,8.9,8.9,APROVADO


### Mesclagem de colunas

In [51]:
# Fazendo um merge entre o dataframe turmas e disciplinas utilizando como base a coluna id_componente
# https://pandas.pydata.org/pandas-docs/stable/merging.html
colunas = ["id_turma","id_docente_interno","codigo","nivel_ensino","nome","unidade_responsavel",
          "situacao_turma","capacidade_aluno","descricao_horario"]

# merge os dois dataframes utilizando como base a coluna id_componente
merge_turmas_disciplinas = pd.merge(turmas_total, disciplinas, on='id_componente')[colunas]

merge_turmas_disciplinas.dropna(inplace=True)

# investigando os dados
merge_turmas_disciplinas.head()

,id_turma,id_docente_interno,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
0,1217099,18225.0,DOL0009,LATO SENSU,Seminário de Simulação (Estudos de caso),DEPARTAMENTO DE OCEANOGRAFIA E LIMNOLOGIA,CONSOLIDADA,26.0,23456T234
1,1217099,18225.0,DOL0009,LATO SENSU,Seminário de Simulação (Estudos de caso),DEPARTAMENTO DE OCEANOGRAFIA E LIMNOLOGIA,CONSOLIDADA,26.0,23456T234
2,1217099,18225.0,DOL0009,LATO SENSU,Seminário de Simulação (Estudos de caso),DEPARTAMENTO DE OCEANOGRAFIA E LIMNOLOGIA,CONSOLIDADA,26.0,23456T234
3,1217100,18695.0,DOL0008,LATO SENSU,: Seminário sobre Políticas Públicas para o De...,DEPARTAMENTO DE OCEANOGRAFIA E LIMNOLOGIA,CONSOLIDADA,26.0,345T2345 6T234 (14/04/2015 - 17/04/2015)
4,57573036,5753068.0,DOL0008,LATO SENSU,: Seminário sobre Políticas Públicas para o De...,DEPARTAMENTO DE OCEANOGRAFIA E LIMNOLOGIA,CONSOLIDADA,2.0,56M12


### Restrição de dados

In [62]:
# filtrando as turmas com código IMD
turmas_total = merge_turmas_disciplinas[merge_turmas_disciplinas.codigo.str.contains("IMD")]

# utilizar apenas disciplinas com código "CONSOLIDADA"
turmas_total = turmas_total[turmas_total.situacao_turma == 'CONSOLIDADA']

# Adaptar o nome da coluna id_docente_interno para id_servidor
turmas_total.rename(columns={'id_docente_interno': 'id_servidor'}, inplace=True)

turmas_total.capacidade_aluno = turmas_total.capacidade_aluno.astype(np.int64)

# Adaptar os tipos de dados em turmas para o merge com o dataframe docentes
turmas_total.id_servidor = turmas_total.id_servidor.astype(np.int64)

# merge os dataframes  turmas_imd e docentes utilizando como base a coluna id_servidor
colunas = ["id_turma", "codigo", "nivel_ensino", "nome_y", "codigo", "nivel_ensino",
          "nome_x","capacidade_aluno", "descricao_horario"]

turmas_nomesprof = pd.merge(turmas_total, docentes, on='id_servidor')[colunas]

# investigando os dados
turmas_nomesprof.head()

,id_turma,codigo,nivel_ensino,nome_y,codigo,nivel_ensino,nome_x,capacidade_aluno,descricao_horario
0,1239374,IMD0019,GRADUAÇÃO,CHARLES ANDRYE GALVAO MADEIRA,IMD0019,GRADUAÇÃO,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI,67,236M1234 (02/02/2015 - 20/06/2015)
1,1239381,IMD0019,GRADUAÇÃO,CHARLES ANDRYE GALVAO MADEIRA,IMD0019,GRADUAÇÃO,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI,66,236M1234 (02/02/2015 - 20/06/2015)
2,57565214,IMD0019,GRADUAÇÃO,CHARLES ANDRYE GALVAO MADEIRA,IMD0019,GRADUAÇÃO,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI,76,246T1234 (27/07/2015 - 19/12/2015)
3,1239374,IMD0019,GRADUAÇÃO,CHARLES ANDRYE GALVAO MADEIRA,IMD0019,GRADUAÇÃO,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI,67,236M1234 (02/02/2015 - 20/06/2015)
4,1239381,IMD0019,GRADUAÇÃO,CHARLES ANDRYE GALVAO MADEIRA,IMD0019,GRADUAÇÃO,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI,66,236M1234 (02/02/2015 - 20/06/2015)


In [20]:
# filtrar apenas as turmas do IMD
notas_imd = notas[notas.id_turma.isin(turmas_nomesprof.id_turma)]

# merge entre os dataframes notas_imd e turmas_imd_nomesprof utilizando como referencia id_turma
notas_imd_merge = pd.merge(notas_imd, turmas_nomesprof, on='id_turma')
notas_imd_merge.descricao.unique()

array(['CANCELADO', 'APROVADO POR NOTA', 'APROVADO', 'REPROVADO',
       'INDEFERIDO', 'DESISTENCIA', 'REPROVADO POR MÉDIA E POR FALTAS',
       'TRANCADO', 'REPROVADO POR NOTA E FALTA', 'REPROVADO POR NOTA',
       'EXCLUIDA', 'MATRICULADO', 'REPROVADO POR FALTAS'], dtype=object)